In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings


### 2  Setup a local LLM


In [2]:
llm = Ollama(model="qwen2.5:7b", request_timeout=120.0)
Settings.llm = llm

### 3  Initialize the MCP client and build the agent


In [3]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client) # you can also pass list of allowed tools

In [4]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

add_data Add new data to the people table using a SQL INSERT query.

    Args:
        query (str): SQL INSERT query following this format:
            INSERT INTO people (name, age, profession)
            VALUES ('John Doe', 30, 'Engineer')
        
    Schema:
        - name: Text field (required)
        - age: Integer field (required)
        - profession: Text field (required)
        Note: 'id' field is auto-generated
    
    Returns:
        bool: True if data was added successfully, False otherwise
    
    Example:
        >>> query = '''
        ... INSERT INTO people (name, age, profession)
        ... VALUES ('Alice Smith', 25, 'Developer')
        ... '''
        >>> add_data(query)
        True
    
read_data Read data from the people table using a SQL SELECT query.

    Args:
        query (str, optional): SQL SELECT query. Defaults to "SELECT * FROM people".
            Examples:
            - "SELECT * FROM people"
            - "SELECT name, age FROM people WHERE ag

In [5]:
SYSTEM_PROMPT = """\
You are an AI assistant for Tool Calling.

Before you help a user, you need to work with tools to interact with Our Database
"""

In [8]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with Our Database software.",
        tools=tools,
        # llm=OpenAI(model="gpt-4"),
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

In [9]:
from llama_index.core.agent.workflow import (
    FunctionAgent, 
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

In [10]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec


mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [11]:
# Run the agent!
while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

User:  hi
Agent:  Hello! How can I assist you today? If you need to perform any operations on the people database, such as adding or reading data, please let me know.
User:  add the following data, gothamchess with age 23
Calling tool add_data with kwargs {'query': "INSERT INTO people (name, age, profession) VALUES ('gothamchess', 23, 'Chess Player')"}
Tool add_data returned meta=None content=[TextContent(type='text', text='true', annotations=None, meta=None)] structuredContent={'result': True} isError=False
Agent:  The data has been added successfully. The record for 'gothamchess' with an age of 23 and the profession 'Chess Player' has been added to the database.
User:  get data
Calling tool read_data with kwargs {}
Tool read_data returned meta=None content=[TextContent(type='text', text='1', annotations=None, meta=None), TextContent(type='text', text='gothamchess', annotations=None, meta=None), TextContent(type='text', text='23', annotations=None, meta=None), TextContent(type='text',